# Revisiting Recommender Systems

The goal of this notebooks it to take a different, more thoughtful approach to building a recommender system (henceforth **recsys**). We will be leveraging more of the Yelp Data this time in order to build more meaningful attributes as a basis for recommendation. 

## Notebook Guide

## Step 1: Getting and examining the Data from disparate datasets

In [1]:
# Imports 
import numpy as np
import scipy
import pandas as pd
import math
import random
import sklearn
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse.linalg import svds
import matplotlib.pyplot as plt

In [2]:
# loading in the data 
sc_reviews = pd.read_csv("food_reviews_sc")
